In Germany seperation of power divides the political power into three branches of government, namely legislature, executive and judiciary.
Therefore the reinging party of a state should theoretically as part of the executive have no influence on judiciary. 

I want to test this and see if there is some correlation between reiging partys and the amount of telecommunication surveillance.

In [1]:
import pandas as pd
import numpy as np

#import sys
#sys.path.insert(0, '..')
#from src.dataProcessing import yearlySum

In [2]:
def yearlySum(xls):
    '''
    This function sums up the data of each year and returns a dataframe with the summed up data.
    '''
    # Get the names of all sheets in the Excel file
    sheet_names = xls.sheet_names

    # Read each sheet into a separate DataFrame and store them in a list
    dfs = [pd.read_excel(xls, sheet_name) for sheet_name in sheet_names]

    # Add a 'Year' column to each DataFrame
    for i in range(len(dfs)):
        dfs[i]['Year'] = sheet_names[i].replace('_surveillance', '')
    
    # Concatenate all DataFrames into one
    data = pd.concat(dfs, ignore_index=True)

    # Convert the 'Year' column to datetime index
    data['Year'] = data['Year'].astype(int)
    data['Year'] = pd.to_datetime(data['Year'], format='%Y')
    data.set_index('Year', inplace=True)

    #We only analyse pargraph 6 as these are the actual crimes
    data_paragraph6 = data[data['paragraph'].isin([1,6])]
    data_paragraph6_int = data_paragraph6.select_dtypes(include=['int'])
    data_yearly = data_paragraph6_int.groupby('Year').sum()
    
    return data_yearly

In [6]:
xls = pd.ExcelFile('../dat/surveillance_data.xlsx')
data = yearlySum(xls)
data = data.drop(columns=['GBA', 'overall'])
data


,paragraph,BW,BY,BE,BB,HB,HH,HE,MV,NI,NW,RP,SL,SN,ST,SH,TH
Year,,,,,,,,,,,,,,,,,
2013-01-01,234,2353,3501,2053,289,330,1175,5313,719,1701,2847,608,115,1166,523,107,338
2016-01-01,65,2076,4271,1845,288,329,1332,5835,324,2262,1927,638,285,1122,473,315,203
2019-01-01,252,2214,4186,1624,181,226,872,5680,365,1813,1132,548,99,654,415,258,285
2020-01-01,88,1988,3770,1364,268,150,949,5230,355,1335,1287,446,99,690,365,986,272
2021-01-01,264,1854,3905,1302,207,129,770,5311,283,1603,1086,526,87,535,397,910,242


Unfortuntly I was not able to find a ready to use dataset on the past governments of germanys states. Therefore I decide to manually take the data from WikiPedia.

https://www.statistik-bw.de/Service/Veroeff/Monatshefte/PDF/Beitrag22_03_09.
https://de.wikipedia.org/wiki/Ergebnisse_der_Landtagswahlen_in_der_Bundesrepublik_Deutschland

In [88]:
import glob
import os
# Get a list of all CSV files
csv_files = glob.glob('../dat/election_results_per_state/*.csv')

# Initialize an empty list to store the dataframes
dfs = []
# Loop through the list of CSV files
for filename in csv_files:
    # Read the CSV file
    df = pd.read_csv(filename, header=0)
    
    # Derive the state name from the filename
    state = os.path.basename(filename).replace('.csv', '')

    # Insert the new column at the third position (index=2)
    df.insert(2, 'State', state)
    
    # Append the dataframe to the list of dataframes
    dfs.append(df)

# Concatenate all dataframes in the list into a single dataframe
df = pd.concat(dfs, ignore_index=True)

df


,Wahltag,Wbt.,State,Grüne,CDU,SPD,FDP/DVP,AfD,NPD,REP,...,FDP,DVU,DZP,Piraten,BiW,SSW,FDP/DPS,FW,CSU,BP
0,27.03.2011,66.3,bw,24.200,39.0,23.1,5.3,0.0,1.00,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.03.2016,70.4,bw,30.300,27.0,12.7,8.3,15.1,0.40,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14.03.2021,63.8,bw,32.600,24.1,11.0,10.5,9.7,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27.09.2009,67.5,bb,5.600,19.8,33.0,NaN,0.0,NaN,NaN,...,7.20,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14.09.2014,47.9,bb,6.200,23.0,31.9,NaN,12.2,NaN,NaN,...,1.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,01.09.2019,61.3,bb,10.800,15.6,26.2,NaN,23.5,NaN,NaN,...,4.10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,13.05.2012,59.6,nw,11.300,26.3,39.1,NaN,0.0,NaN,NaN,...,8.60,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,14.05.2017,65.2,nw,6.400,33.0,31.2,NaN,7.4,NaN,NaN,...,12.60,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15.05.2022,55.5,nw,18.200,35.7,26.7,NaN,5.4,NaN,NaN,...,5.90,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,30.08.2009,56.2,th,6.200,31.2,18.5,NaN,0.0,NaN,NaN,...,7.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
by= pd.read_csv('../dat/election_results_per_state/by.csv', header=0)
by

,Wahltag,Wbt.,CSU,FW,AfD,Grüne,SPD,FDP,BP,NPD,Übrige
0,28.09.2008,58.1,43.4,10.2,0.0,9.4,18.6,8.0,1.1,1.2,8.1
1,15.09.2013,63.9,47.7,9.0,0.0,8.6,20.6,3.3,2.1,0.6,8.1
2,14.10.2018,72.4,37.2,11.6,10.2,17.6,9.7,5.1,1.7,0.0,7.0
3,8.10.2023,73.3,37.0,15.8,14.6,14.4,8.4,3.0,0.9,0.0,5.9


In [90]:
df.columns

Index(['Wahltag', 'Wbt.', 'State', 'Grüne', 'CDU', 'SPD', 'FDP/DVP', 'AfD',
       'NPD', 'REP', 'Übrige', 'Linke', 'BVB/FW', 'FDP', 'DVU', 'DZP',
       'Piraten', 'BiW', 'SSW', 'FDP/DPS', 'FW', 'CSU', 'BP'],
      dtype='object')

In [91]:
df.loc[df['State'] == 'by']

,Wahltag,Wbt.,State,Grüne,CDU,SPD,FDP/DVP,AfD,NPD,REP,...,FDP,DVU,DZP,Piraten,BiW,SSW,FDP/DPS,FW,CSU,BP
51,28.09.2008,58.1,by,9.4,NaN,18.6,NaN,0.0,1.2,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,10.2,43.4,1.1
52,15.09.2013,63.9,by,8.6,NaN,20.6,NaN,0.0,0.6,NaN,...,3.3,NaN,NaN,NaN,NaN,NaN,NaN,9.0,47.7,2.1
53,14.10.2018,72.4,by,17.6,NaN,9.7,NaN,10.2,0.0,NaN,...,5.1,NaN,NaN,NaN,NaN,NaN,NaN,11.6,37.2,1.7
54,8.10.2023,73.3,by,14.4,NaN,8.4,NaN,14.6,0.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,15.8,37.0,0.9


In [92]:
len(df['State'].unique()) == 16


True

In [93]:
df.fillna(0, inplace=True)
# Create a new column 'total' that contains the sum of all values in each row
df['total'] = df.iloc[:, 3:].sum(axis=1)

# Create a new column 'check' that checks if 'total' is equal to 1
df['check'] = df['total'] == 1
df['total']

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [94]:
df.dtypes


Wahltag     object
Wbt.       float64
State       object
Grüne      float64
CDU        float64
SPD        float64
FDP/DVP    float64
AfD        float64
NPD        float64
REP        float64
Übrige     float64
Linke      float64
BVB/FW      object
FDP        float64
DVU        float64
DZP        float64
Piraten    float64
BiW        float64
SSW        float64
FDP/DPS    float64
FW         float64
CSU        float64
BP         float64
dtype: object

In [ ]:
df['Übrige']

0       6.3
1       5.9
2      12.1
3       4.4
4       6.6
5       5.1
6     12.2⁠
7       4.5
8       6.0
9       9.1
10      7.1
11      5.4
12      8.4
13      7.4
14     12.0
15      8.7
16      6.9
17      4.9
18      5.3
19      6.8
20      5.5
21    10.6⁠
22      3.4
23      4.1
24      4.6
25    11.9⁠
26      3.3
27      9.9
28      6.8
29      5.2
30      9.2
31      2.9
32      8.6
33      6.3
34      9.0
35      5.4
36      4.2
37      6.1
38      6.1
39      5.2
40      8.2
41     11.4
42      8.9
43     10.1
44      2.4
45      3.9
46      3.5
47      5.6
48      5.6
49      5.6
50      7.6
51      8.1
52      8.1
53      7.0
54      5.9
Name: Übrige, dtype: object

In [ ]:
df['Übrige'].astype(float)

ValueError: could not convert string to float: '12.2\u2060'